# Megaline plan recomendator

Contents <a name="back"></a>

1. [Introduction](#introduction)
2. [Data Loading and Inspection]
3. [Model training]
    1. [Splitting the data into sets]
    2. [Tree model]
    3. [Forest model]
    4. [Logistic Regression model]
    5. [Quality check using the test set]
    6. [Sanity check]
4. [Conclusion]